# Saturation Limits for Direct Imaging

For each NIRCam filter, find the saturation limit for a 2-frame (CDS) ramp.

This notebook cycles over all SW and LW filters to find saturation limits for various detector readout modes:
 * Full Frame
 * Window
  * SUB64
  * SUB160
  * SUB320
  * SUB400
  * SUB640
 * Stripe modes
  * STRIPE64
  * STRIPE128
  * STRIPE256
 
Saturation limits for both NIRCam modules (A and B) are calculated.

Results are returned an AstroPy Table of the results, saved in a dictionary `tbls_dict`.

In [1]:
## Standard imports

# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'none'

# seaborn package for making pretty plots, but not necessary
try:
    import seaborn as sns
    params =   {'xtick.direction': 'in', 'ytick.direction': 'in', 'font.family': ['serif'],
                'text.usetex': True, 'text.latex.preamble': ['\usepackage{gensymb}']}
    sns.set_style("ticks", params)
except ImportError:
    print('Seaborn module is not installed. Not strictly required, but is a useful package!')
    
from IPython.display import display, Latex, clear_output

In [11]:
# pyNRC and associates
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import (webbpsf, poppy, pix_noise, S)
from astropy.table import Table

pynrc.setup_logging('WARNING', verbose=False)

In [7]:
# Filters
filt_wide_sw = ['F070W', 'F090W', 'F115W', 'F150W', 'F150W2', 'F200W']
filt_wide_lw = ['F277W', 'F322W2', 'F356W', 'F444W']

filt_med_sw = ['F140M', 'F162M', 'F182M', 'F210M']
filt_med_lw = ['F250M', 'F300M', 'F335M', 'F360M', 'F410M', 'F430M', 'F460M', 'F480M']

filt_narr_sw = ['F164N', 'F187N', 'F212N']
filt_narr_lw = ['F323N', 'F405N', 'F466N', 'F470N']

filt_all_sw = filt_wide_sw + filt_med_sw + filt_narr_sw
filt_all_lw = filt_wide_lw + filt_med_lw + filt_narr_lw
filt_all = filt_all_sw + filt_all_lw

In [43]:
# NIRCam readout modes
full   = {'wind_mode':'FULL',   'xpix':2048, 'ypix':2048}
# Window subarrays
sub64  = {'wind_mode':'WINDOW', 'xpix':64,   'ypix':64}
sub160 = {'wind_mode':'WINDOW', 'xpix':160,  'ypix':160}
sub320 = {'wind_mode':'WINDOW', 'xpix':320,  'ypix':320}
sub400 = {'wind_mode':'WINDOW', 'xpix':400,  'ypix':400}
sub640 = {'wind_mode':'WINDOW', 'xpix':640,  'ypix':640}
# Stripe subarrays
stp64  = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':64}
stp128 = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':128}
stp256 = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':256}

subarray_settings = [full, sub64, sub160, sub320, sub400, sub640, stp64, stp128, stp256]

# Header names for each readout mode
head = []
head2 = []
for d in subarray_settings:
    if 'FULL' in d['wind_mode']:
        name = 'FULL'
        name2 = name
    elif 'WINDOW' in d['wind_mode']:
        name = 'SUB' + str(d['ypix'])
        name2 = name
    elif 'STRIPE' in d['wind_mode']:
        name = 'STRIPE' + str(d['ypix'])
        name2 = 'STRP' + str(d['ypix'])
    head.append(name)
    head2.append(name2)

In [53]:
nfilt = len(filt_all)
nsub  = len(subarray_settings)
sat_lims = np.zeros([nfilt,nsub])

# Module A
module = 'A'

# We want the K-Band limiting magnitude for a G2V star
sp = nrc_utils.stellar_spectrum('G2V')
bp_lim = S.ObsBandpass('johnson,k')
bp_lim.name = 'K-Band'


# Loop through each subrray mode for a given filter and print the CDS saturation limit
modules = ['A','B']
tbls_dict = {}
for mod in modules:
    print('Module {} \n{} saturation magnitudes assuming {} star\n'.format(mod,bp_lim.name, sp.name))
    print('{:<6}'.format('Filter') + ' '.join('{:>7}'.format(t) for t in head2))
    for i,filt in enumerate(filt_all):
        for j,subset in enumerate(subarray_settings):
            nrc = pynrc.NIRCam(filt, read_mode='RAPID', ngroup=2, module=mod, **subset)
            lim = nrc.sat_limits(sp=sp, bp_lim=bp_lim, units='vegamag')
            sat_lims[i,j] = lim['satmag']
        print('{:<6}'.format(filt) + ' '.join('{: 7.2f}'.format(s) for s in sat_lims[i,:]))
        
    print("")
    # Create astropy Table to hold data
    tbl = Table(sat_lims, names=head)
    for k in tbl.keys():
        tbl[k].format = '6.2f'

    # Add filter column and reorder to front
    tbl['Filter'] = filt_all
    col_order = ['Filter'] + head
    tbl = tbl[col_order]
    
    tbls_dict[mod] = tbl

Module A 
K-Band saturation magnitudes assuming G2V star

Filter   FULL   SUB64  SUB160  SUB320  SUB400  SUB640  STRP64 STRP128 STRP256
F070W   14.92    9.09   10.95   12.41   12.89   13.89   11.17   11.91   12.66
F090W   15.64    9.81   11.68   13.14   13.61   14.62   11.89   12.64   13.39
F115W   15.71    9.89   11.75   13.21   13.69   14.69   11.97   12.71   13.46
F150W   15.62    9.79   11.65   13.11   13.59   14.59   11.87   12.61   13.36
F150W2  16.86   11.03   12.90   14.36   14.83   15.84   13.11   13.86   14.61
F200W   15.06    9.23   11.09   12.55   13.03   14.04   11.31   12.06   12.80
F140M   14.86    9.03   10.89   12.35   12.83   13.83   11.11   11.85   12.60
F162M   14.64    8.81   10.67   12.13   12.61   13.62   10.89   11.64   12.38
F182M   14.60    8.78   10.64   12.10   12.57   13.58   10.86   11.60   12.35
F210M   13.93    8.10    9.97   11.43   11.90   12.91   10.18   10.93   11.68
F164N   12.24    6.41    8.28    9.74   10.21   11.22    8.49    9.24    9.99
F187N 

In [55]:
print(tbls_dict['A'])

Filter  FULL  SUB64  SUB160 SUB320 SUB400 SUB640 STRIPE64 STRIPE128 STRIPE256
------ ------ ------ ------ ------ ------ ------ -------- --------- ---------
 F070W  14.89   9.07  10.93  12.39  12.86  13.87    11.15     11.89     12.64
 F090W  15.67   9.84  11.70  13.16  13.64  14.64    11.92     12.66     13.41
 F115W  15.70   9.87  11.73  13.19  13.67  14.68    11.95     12.70     13.44
 F150W  15.61   9.79  11.65  13.11  13.58  14.59    11.87     12.61     13.36
F150W2  16.86  11.03  12.89  14.35  14.83  15.84    13.11     13.86     14.60
 F200W  15.05   9.22  11.09  12.55  13.02  14.03    11.30     12.05     12.80
 F140M  14.86   9.04  10.90  12.36  12.83  13.84    11.12     11.86     12.61
 F162M  14.64   8.81  10.68  12.14  12.61  13.62    10.89     11.64     12.39
 F182M  14.60   8.78  10.64  12.10  12.57  13.58    10.86     11.60     12.35
 F210M  13.91   8.09   9.95  11.41  11.88  12.89    10.17     10.91     11.66
   ...    ...    ...    ...    ...    ...    ...      ...       

In [56]:
print(tbls_dict['B'])

Filter  FULL  SUB64  SUB160 SUB320 SUB400 SUB640 STRIPE64 STRIPE128 STRIPE256
------ ------ ------ ------ ------ ------ ------ -------- --------- ---------
 F070W  14.89   9.07  10.93  12.39  12.86  13.87    11.15     11.89     12.64
 F090W  15.67   9.84  11.70  13.16  13.64  14.64    11.92     12.66     13.41
 F115W  15.70   9.87  11.73  13.19  13.67  14.68    11.95     12.70     13.44
 F150W  15.61   9.79  11.65  13.11  13.58  14.59    11.87     12.61     13.36
F150W2  16.86  11.03  12.89  14.35  14.83  15.84    13.11     13.86     14.60
 F200W  15.05   9.22  11.09  12.55  13.02  14.03    11.30     12.05     12.80
 F140M  14.86   9.04  10.90  12.36  12.83  13.84    11.12     11.86     12.61
 F162M  14.64   8.81  10.68  12.14  12.61  13.62    10.89     11.64     12.39
 F182M  14.60   8.78  10.64  12.10  12.57  13.58    10.86     11.60     12.35
 F210M  13.91   8.09   9.95  11.41  11.88  12.89    10.17     10.91     11.66
   ...    ...    ...    ...    ...    ...    ...      ...       